In [1]:
import numpy as np
import pandas as pd
from make_cache import main, decryption
num_threads = 8
max_n_plus = 5
seed = np.random.randint(10000)
# seed = 2062
# seed = 1986
# seed = 9852
for n_plus in range(1, max_n_plus + 1):
    main(n_plus,num_threads,len_x=40, log_interval=3, seed=seed)

2025-04-03 16:31:15,329 : numba=0.60.0, numpy=2.0.2
2025-04-03 16:31:15,329 : OPT=_OptLevel(3), THREADING_LAYER=default
2025-04-03 16:31:15,329 : USING_SVML=False, ENABLE_AVX=True, DISABLE_JIT=0
2025-04-03 16:31:15,330 : max_op = 1  ,  date : 2025-04-03 16:31:15.330184
2025-04-03 16:31:15,330 : use cores : 8
2025-04-03 16:31:15,330 : making random_x
2025-04-03 16:31:15,330 :    seed = 371, loop = 100000, upper = 0.5, lower = 1.5
2025-04-03 16:31:15,811 : random_x corrcoef = 3.3411669768254657e-06
2025-04-03 16:31:16,209 : make_before_similar_num_list
2025-04-03 16:31:16,209 :    using Memory size =  195.0 MB
2025-04-03 16:31:16,209 :    Memory size of numpy array = 0.0 M bytes +alpha (1data=1280 bytes, loop=2)
2025-04-03 16:31:19,341 :       0/2  (221.1 MB)  0:00:03.001206 : inf
2025-04-03 16:31:20,918 :    time : 0:00:04.707772
2025-04-03 16:31:20,918 : make_unique_equations
2025-04-03 16:31:20,918 :    n_op1 = 0, n_op2 = 0
2025-04-03 16:31:22,786 :       make_unique_equations_thread


In [2]:
import sys

# 新しいオブジェクトを作成
a = []
print(sys.getrefcount(a) - 1)  # 1 (-1 はgetrefcount自体による参照を除外)

# 別の参照を作成
b = a
print(sys.getrefcount(a) - 1)  # 2
print(sys.getrefcount(b) - 1)

# 参照を削除
del b
print(sys.getrefcount(a) - 1)  # 1

1
2
2
1


In [ ]:
[2, 10, 80, 911, 13471, 251705, 5561536]
630:281
10208:3263
201552:50153

SyntaxError: illegal target for annotation (3326823552.py, line 2)

1 : 201838:49888   
4 : 201838:49888   
8 : 201838:49888   

In [9]:
np.random.randint(1,4,5)

array([3, 3, 2, 1, 3])

In [ ]:
import numpy as np
n_selected_indexes = 7
i = 3
use = np.zeros(i, dtype="int")
arr_max = np.array([6, 3, 7], dtype="int")
arr_use = []
while True:
    arr_use.append(use.copy())
    done_plus_one = False
    for l in range(i - 1, -1, -1):
        if use[l] != arr_max[l] - 1:
            use[l] += 1
            done_plus_one = True
            break
        else:
            use[l] = 0
    if not done_plus_one:
        break
print(arr_use)

In [6]:
import numpy as np
n_selected_indexes = 7
i = 3
use = np.zeros(i, dtype="int")
arr_max = np.array([6, 3, 7], dtype="int")
arr_use = []
while True:
    arr_use.append(use.copy())
    done_plus_one = False
    for l in range(i - 1, -1, -1):
        if use[l] != arr_max[l] - 1:
            use[l] += 1
            done_plus_one = True
            break
        else:
            use[l] = 0
    if not done_plus_one:
        break
print(arr_use)

[array([0, 0, 0]), array([0, 0, 1]), array([0, 0, 2]), array([0, 0, 3]), array([0, 0, 4]), array([0, 0, 5]), array([0, 0, 6]), array([0, 1, 0]), array([0, 1, 1]), array([0, 1, 2]), array([0, 1, 3]), array([0, 1, 4]), array([0, 1, 5]), array([0, 1, 6]), array([0, 2, 0]), array([0, 2, 1]), array([0, 2, 2]), array([0, 2, 3]), array([0, 2, 4]), array([0, 2, 5]), array([0, 2, 6]), array([1, 0, 0]), array([1, 0, 1]), array([1, 0, 2]), array([1, 0, 3]), array([1, 0, 4]), array([1, 0, 5]), array([1, 0, 6]), array([1, 1, 0]), array([1, 1, 1]), array([1, 1, 2]), array([1, 1, 3]), array([1, 1, 4]), array([1, 1, 5]), array([1, 1, 6]), array([1, 2, 0]), array([1, 2, 1]), array([1, 2, 2]), array([1, 2, 3]), array([1, 2, 4]), array([1, 2, 5]), array([1, 2, 6]), array([2, 0, 0]), array([2, 0, 1]), array([2, 0, 2]), array([2, 0, 3]), array([2, 0, 4]), array([2, 0, 5]), array([2, 0, 6]), array([2, 1, 0]), array([2, 1, 1]), array([2, 1, 2]), array([2, 1, 3]), array([2, 1, 4]), array([2, 1, 5]), array([2,

In [6]:
n = 3
use = np.zeros(n, dtype="int")
arr_max = np.array([6, 3, 7], dtype="int")
loop = 1
for i in arr_max:
    loop *= i
for i in range(loop):
    cp_i = i
    for j in range(n):
        use[j] = cp_i % arr_max[j]
        cp_i //= arr_max[j]
    print(use)

[0 0 0]
[1 0 0]
[2 0 0]
[3 0 0]
[4 0 0]
[5 0 0]
[0 1 0]
[1 1 0]
[2 1 0]
[3 1 0]
[4 1 0]
[5 1 0]
[0 2 0]
[1 2 0]
[2 2 0]
[3 2 0]
[4 2 0]
[5 2 0]
[0 0 1]
[1 0 1]
[2 0 1]
[3 0 1]
[4 0 1]
[5 0 1]
[0 1 1]
[1 1 1]
[2 1 1]
[3 1 1]
[4 1 1]
[5 1 1]
[0 2 1]
[1 2 1]
[2 2 1]
[3 2 1]
[4 2 1]
[5 2 1]
[0 0 2]
[1 0 2]
[2 0 2]
[3 0 2]
[4 0 2]
[5 0 2]
[0 1 2]
[1 1 2]
[2 1 2]
[3 1 2]
[4 1 2]
[5 1 2]
[0 2 2]
[1 2 2]
[2 2 2]
[3 2 2]
[4 2 2]
[5 2 2]
[0 0 3]
[1 0 3]
[2 0 3]
[3 0 3]
[4 0 3]
[5 0 3]
[0 1 3]
[1 1 3]
[2 1 3]
[3 1 3]
[4 1 3]
[5 1 3]
[0 2 3]
[1 2 3]
[2 2 3]
[3 2 3]
[4 2 3]
[5 2 3]
[0 0 4]
[1 0 4]
[2 0 4]
[3 0 4]
[4 0 4]
[5 0 4]
[0 1 4]
[1 1 4]
[2 1 4]
[3 1 4]
[4 1 4]
[5 1 4]
[0 2 4]
[1 2 4]
[2 2 4]
[3 2 4]
[4 2 4]
[5 2 4]
[0 0 5]
[1 0 5]
[2 0 5]
[3 0 5]
[4 0 5]
[5 0 5]
[0 1 5]
[1 1 5]
[2 1 5]
[3 1 5]
[4 1 5]
[5 1 5]
[0 2 5]
[1 2 5]
[2 2 5]
[3 2 5]
[4 2 5]
[5 2 5]
[0 0 6]
[1 0 6]
[2 0 6]
[3 0 6]
[4 0 6]
[5 0 6]
[0 1 6]
[1 1 6]
[2 1 6]
[3 1 6]
[4 1 6]
[5 1 6]
[0 2 6]
[1 2 6]
[2 2 6]
[3 2 6]
[4 2 6]


In [ ]:
import numpy as np
i = 3
n_selected_indexes = 5
use = np.arange(i)
while True:
    print(use)
    done_plus_one = False
    for l in range(i - 1, -1, -1):
        if use[l] != n_selected_indexes - (i - l):
            use[l] += 1
            done_plus_one = True
            break
        else:
            for m in range(l - 1, -1, -1):
                if use[m] != n_selected_indexes - (i - m):
                    use[l] = use[m] + 1 + (l - m)
                    break
    if not done_plus_one:
        break

In [7]:
import numpy as np
i = 3
n_selected_indexes = 5
use = np.arange(i)
while True:
    print(use)
    done_plus_one = False
    for l in range(i - 1, -1, -1):
        if use[l] != n_selected_indexes - (i - l):
            use[l] += 1
            done_plus_one = True
            break
        else:
            for m in range(l - 1, -1, -1):
                if use[m] != n_selected_indexes - (i - m):
                    use[l] = use[m] + 1 + (l - m)
                    break
    if not done_plus_one:
        break

[0 1 2]
[0 1 3]
[0 1 4]
[0 2 3]
[0 2 4]
[0 3 4]
[1 2 3]
[1 2 4]
[1 3 4]
[2 3 4]


In [ ]:
import numpy as np
i = 3
n_selected_indexes = 5
num_threads = 3
use = np.arange(i)
while True:
    print(use)
    remaining_num = num_threads
    done_plus = False
    for l in range(i - 1, -1, -1):
        if use[l] + remaining_num < n_selected_indexes - (i - l) + 1:
            use[l] += remaining_num
            remaining_num = 0
            done_plus = True
            break
        else:
            for m in range(l - 1, -1, -1):
                if use[m] != n_selected_indexes - (i - m):
                    use[l] = use[m] + 1 + (l - m)
                    break
    if not done_plus:
        break

2, 10, 80, 911, 13471, 251726   
1 : 6:4   
2 : 53:27   
3 : 636:275   
4 : 10241:3230   
5 : 201838:49888   

In [ ]:
#np.save("arr_len", np.array([2],dtype="int64"))
#np.save("operator_0",np.array([[1],[0]],dtype="int8"))
#np.savez(f"check_change_x_tot_0", np.full((2, 2, 2), -100, dtype="int8"))

In [ ]:
np.load("operator_0.npy")

In [ ]:
np.arange(2,dtype="int8")[::-1].reshape((-1,1))

In [ ]:
np.load("check_change_x_tot_0.npz")["arr_0"]

In [ ]:
import numpy as np
from numba import njit, prange
from numba_progress.numba_atomic import atomic_add, atomic_xchg,atomic_min

@njit()
def fibo(n):
    a, b = 0, 1
    fibolist=np.zeros(n)
    for i in range(n):
        a, b = b, a+b
        fibolist[i]=a


@njit(parallel=True, error_model="numpy")
def func(loop, target_num, num_threads = 8):
    weight = 100
    counter = np.zeros(num_threads, dtype="int64")
    checked = np.zeros(1, dtype="int8")
    for thread_id in prange(num_threads):
        for i in range(thread_id, loop, num_threads):
            if checked[0] == 1:
                break
            for j in range(weight):
                fibo(10000)
            counter[thread_id] = i
            if target_num == i:
                atomic_add(checked,0,1)
                break
    return counter
loop = 100000
target_num = 30
func(loop, target_num)

In [ ]:
import numpy as np
from numba import njit, prange
from numba_progress.numba_atomic import atomic_add, atomic_xchg,atomic_min

@njit()
def fibo(n):
    a, b = 0, 1
    fibolist=np.zeros(n)
    for i in range(n):
        a, b = b, a+b
        fibolist[i]=a


@njit(parallel=True, error_model="numpy")
def func(loop, target_num, num_threads = 8):
    weight = 100
    counter = np.zeros(num_threads, dtype="int64")
    checked = np.zeros(1, dtype="int8")
    for i in prange(loop):
        if checked[0] == 0:
            for j in range(weight):
                fibo(10000)
            counter[i % num_threads] = i
            if target_num == i:
                atomic_add(checked,0,1)
    return counter, checked
loop = 100000
target_num = 30
func(loop, target_num)

In [ ]:
np.random.seed(100)
max_op = 5
len_x = 300

#logger.info("making random_x")
save_random_x = np.random.uniform(0.3, 10, (max_op + 2, len_x))
save_corrcoef = np.max(np.abs(np.corrcoef(save_random_x[1:]) - np.eye(max_op + 1)))
for i in range(100000):
    random_x = np.random.uniform(0.3, 10, (max_op + 2, len_x))
    corrcoef = np.max(np.abs(np.corrcoef(random_x[1:]) - np.eye(max_op + 1)))
    if save_corrcoef > corrcoef:
        save_corrcoef = corrcoef
        save_random_x = random_x
random_x = save_random_x.copy()
random_x[0] = 1

In [ ]:
from numba import njit
@njit
def func(max_op, len_x):
    save_random_x = np.random.uniform(0.3, 10, (max_op + 2, len_x))
    save_corrcoef = np.max(np.abs(np.corrcoef(save_random_x[1:]) - np.eye(max_op + 1)))
    for _ in range(100000):
        random_x = np.random.uniform(0.3, 10, (max_op + 2, len_x))
        corrcoef = np.max(np.abs(np.corrcoef(random_x[1:]) - np.eye(max_op + 1)))
        if save_corrcoef > corrcoef:
            save_corrcoef = corrcoef
            save_random_x = random_x
    random_x = save_random_x.copy()
    random_x[0] = 1
    print(save_corrcoef)
max_op = 5
len_x = 300
func(max_op, len_x)

In [ ]:
func(max_op, len_x)

In [ ]:
np.random.uniform(-10, 10)

In [ ]:
from numba import njit
import numpy as np

@njit
def corrcoef(x_1, x_2):
    mean_1, mean_2 = np.mean(x_1), np.mean(x_2)
    x_len = x_1.shape[0]
    sum_1, sum_2, cov = 0.0, 0.0, 0.0
    for i in range(x_len):
        sum_1 += (x_1[i] - mean_1)**2
        sum_2 += (x_2[i] - mean_2)**2
        cov += (x_1[i] - mean_1) * (x_2[i] - mean_2)
    return np.abs(cov / np.sqrt(sum_1 * sum_2))

@njit
def func(max_op, len_x, seed=-100, loop=100000, upper=0.3, lower=10):
    if seed != -100:
        np.random.seed(seed)
    save_random_x = np.random.uniform(upper,lower , (max_op + 2, len_x))
    save_corrcoef = 0
    for i in range(1, max_op + 1):
        for j in range(i + 1, max_op + 2):
            new_corrcoef = corrcoef(save_random_x[i], save_random_x[j])
            if new_corrcoef > save_corrcoef:
                save_corrcoef = new_corrcoef
    for _ in range(loop):
        random_x = np.random.uniform(upper, lower, (max_op + 2, len_x))
        now_corrcoef = 0
        for i in range(1, max_op + 1):
            for j in range(i + 1, max_op + 2):
                new_corrcoef = corrcoef(random_x[i], random_x[j])
                if new_corrcoef > now_corrcoef:
                    now_corrcoef = new_corrcoef
        if save_corrcoef > now_corrcoef:
            save_corrcoef = now_corrcoef
            save_random_x = random_x
    save_random_x[0] = 1
    return save_corrcoef, save_random_x

In [ ]:
max_op = 5
len_x = 300
func(max_op, len_x,seed=-100, loop=1000000, upper=-10, lower=10)[0]

In [6]:
len_random_x = 50
max_op = 7
mem_size_per_1data = (  # Byte
    len_random_x * 8  # save_similar_num_list
    + 1 * 8  # head_save_similar_num_list
    + 1  # TF_list
    + 1  # check_exist_TF
    + (2 * max_op + 1)  # save_equation_list
    + 5 * 8  # save_base_eq_id_list
    + max_op  # save_back_change_pattern
    + (max_op * (max_op + 1))  # save_check_change_x_tot
    + (max_op * (max_op + 1))  # save_check_change_x_ones
)
mem_size_per_1data*129316244//1000000000

75

In [9]:
import platform
import resource

def get_maxrss() -> float:
    system = platform.system()
    r = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
    if system == "Darwin":
        return r / 1024 / 1024  # bytes on MacOS
    return r / 1024  # kilobytes on Linux
get_maxrss()

62.484375

In [4]:
import numpy as np
from sys import getsizeof
import resource
start_using_mem = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
print(start_using_mem)
a = np.random.random((1000,10000))
print(getsizeof(a), start_using_mem + getsizeof(a))
print(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)

156876800
80000128 236876928
236863488


In [6]:
import resource
def str_using_mem():
    using_mem = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
    return f"{using_mem /(1024**2):.1f} MB"
str_using_mem()

'225.9 MB'

In [2]:
import numpy as np
a = np.random.random((1000,10000))
a.__sizeof__()

80000128

In [5]:
import numpy as np
from numba import njit
@njit(error_model="numpy")
def numba_del():
    a = np.random.random((1000,10000))
    del a
a = np.random.random((1000,10000))
print(a.__sizeof__())
numba_del()
# print(a.__sizeof__())

80000128


CompilerError: Failed in nopython mode pipeline (step: analyzing bytecode)
[1mIllegal IR, del found at: del a
[1m
File "../../../../../../../../var/folders/tj/kgkr2hgj2csbcqdhnxjz2_nh0000gn/T/com.apple.shortcuts.mac-helper/ipykernel_16659/3546667526.py", line 6:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m